## **Quick and easy use of KA-Search**

For an easy search without much configuration, use the EasySearch module. This module only allows you to run a single query at a time on a single region. For running more queries and searching for more regions simultaneously, look at using KA-Search with more configuration.

In [1]:
from kasearch import EasySearch

In [2]:
query = 'QVQLVQSGPEVKKPGTSVKVSCKASGFTFTTSAVQWVRQARGQRLEWIGWIVVGSGNTNYAQKFQERVTITRDMSTTTAYMELSSLRSEDTAVYFCAAPHCNSTSCYDAFDIWGQGTMVTVSS'

results = EasySearch(
    query,                         # Single sequence to search
    keep_best_n=5,                 # Number of closest matches to return
    #database_path                 # Database to search. If not set, oas-aligned-tiny will be automatically downloaded and used
    allowed_chain='Any',           # Chains to search, either 'Heavy', 'Light' or 'Any'
    allowed_species='Any',         # Species to search
    regions=['cdr3'],              # Region to search, either 'whole', 'cdrs', 'cdr3' or a user-specified region
    length_matched=[True],         # Restrict the search to antibodies with the same number of amino acids in the selected region
    n_jobs=10,                     # Number of threads to use (jax will use any available)
    include_ends=True,
)

results

,sequence,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call,d_call,...,Longitudinal,Age,Disease,Subject,Vaccine,Chain,Unique sequences,Total sequences,Isotype,Identity
0,ACCTATGAAACACCTGTGGTTCTTCCTCCTCCTGGTGGCAGCTCCC...,H,F,T,F,T,T,T,IGHV4-4*02,IGHD2-2*01,...,no,22,None,Subject-HIP2,None,Heavy,4474418,21326500,IGHM,0.8125
1,ACATGGTTCCTCTTTGTGGTGGCAGCAGCTACAGGTGTCCAGTCCC...,H,F,T,F,T,T,T,IGHV1-69*12,IGHD2-2*01,...,no,29,None,Subject-HIP3,None,Heavy,4384076,25225992,IGHM,0.8125
2,GGTGGGAGCAGCAACAAGAGCCCACTCCCAGGTGCAGCTGGTGCAA...,H,F,T,F,T,F,T,IGHV7-4-1*02,IGHD2-2*01,...,no,no,None,A030,None,Heavy,174608,821620,IGHM,0.8125
3,GGGTTTTCCTTGTTGCTATTTTAAAAGGTGTCCAATGTGAGGTGCA...,H,F,T,F,T,F,T,IGHV3-64*01,IGHD2-2*01,...,no,no,None,A007,None,Heavy,265055,1206161,IGHM,0.8125
4,TGTTCTCTAAGCAGTGGTATCAACGCAGAGTACATGGGGATGCTTT...,H,F,T,F,T,T,T,IGHV4-39*01,IGHD2-2*01,...,no,no,None,Subject-CB1g,None,Heavy,11894,25712,IGHG,0.8125


------------------
## **KA-Search with more configuration**

In [3]:
from kasearch import AlignSequences, SearchDB, PrepareDB

### **Align query sequences**

Sequences to search with needs to be aligned to the KA-Search alignment as described in Olsen et al, 2022.

In [4]:
raw_queries = [
    'QVQLVQSGPEVKKPGTSVKVSCKASGFTFTTSAVQWVRQARGQRLEWIGWIVVGSGNTNYAQKFQERVTITRDMSTTTAYMELSSLRSEDTAVYFCAAPHCNSTSCYDAFDIWGQGTMVTVSS',
    'EVQLVQSGAEVKKPGASVKISCKASGYTFSEYFLHWVRQAPGTGLEWVGLVDPEDGEVIYTQKFQDRVTITADTSTDTAYMEQRSLRSENTAFYYYATGRCSGSVCQPEFDYWGQGVLVTVSS'
]

aligned_seqs = AlignSequences(
    allowed_species=['Human', 'Mouse'], # Species to use for numbering (human and mouse is default).
    n_jobs=1                            # Allocated number for jobs/threads for the search.
)(raw_queries)                          # Sequences as strings to align.

aligned_seqs

array([[81, 86, 81,  0, 76, 86, 81, 83, 71, 80,  0, 69, 86, 75, 75, 80,
        71, 84, 83, 86, 75, 86, 83, 67, 75, 65, 83, 71, 70, 84, 70,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 84, 84, 83, 65, 86, 81,  0, 87,
        86, 82, 81,  0, 65,  0, 82,  0, 71,  0, 81,  0,  0, 82,  0, 76,
        69,  0, 87, 73, 71, 87, 73, 86, 86, 71,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0, 83, 71, 78, 84, 78, 89,  0,  0, 65,  0,  0,
        81,  0,  0, 75, 70,  0,  0, 81,  0,  0,  0, 69, 82, 86, 84, 73,
        84, 82,  0, 68,  0,  0,  0, 77,  0, 83,  0,  0, 84, 84,  0,  0,
         0,  0, 84,  0, 65, 89, 77, 69, 76, 83, 83, 76, 82, 83,  0, 69,
        68, 84, 65, 86, 89, 70, 67, 65, 65, 80, 72, 67, 78, 83, 84,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 83, 67, 89, 68, 65, 70, 68, 73, 87, 71,  0, 81,
        71, 84, 77, 86, 84, 86, 83, 83],
       [69, 86, 81,  0, 76, 86, 81, 83, 71, 65,  0, 69, 86, 75, 75, 80,
        71, 65, 83, 86,

### Canonical alignment
The unique positions allowed in the canonical alignment can be viewed with the following 

In [5]:
from kasearch import canonical_numbering
print(canonical_numbering)

['1 ', '2 ', '3 ', '3A', '4 ', '5 ', '6 ', '7 ', '8 ', '9 ', '10 ', '11 ', '12 ', '13 ', '14 ', '15 ', '16 ', '17 ', '18 ', '19 ', '20 ', '21 ', '22 ', '23 ', '24 ', '25 ', '26 ', '27 ', '28 ', '29 ', '30 ', '31 ', '32 ', '32A', '32B', '33C', '33B', '33A', '33 ', '34 ', '35 ', '36 ', '37 ', '38 ', '39 ', '40 ', '40A', '41 ', '42 ', '43 ', '44 ', '44A', '45 ', '45A', '46 ', '46A', '47 ', '47A', '48 ', '48A', '48B', '49 ', '49A', '50 ', '51 ', '51A', '52 ', '53 ', '54 ', '55 ', '56 ', '57 ', '58 ', '59 ', '60 ', '60A', '60B', '60C', '60D', '61E', '61D', '61C', '61B', '61A', '61 ', '62 ', '63 ', '64 ', '65 ', '66 ', '67 ', '67A', '67B', '68 ', '68A', '68B', '69 ', '69A', '69B', '70 ', '71 ', '71A', '71B', '72 ', '73 ', '73A', '73B', '74 ', '75 ', '76 ', '77 ', '78 ', '79 ', '80 ', '80A', '81 ', '81A', '81B', '81C', '82 ', '82A', '83 ', '83A', '83B', '84 ', '85 ', '85A', '85B', '85C', '85D', '86 ', '86A', '87 ', '88 ', '89 ', '90 ', '91 ', '92 ', '93 ', '94 ', '95 ', '96 ', '96A', '97 ', '

--------------
### **Initiate search class**

#### Database to search against
- If no database path is given, a small OAS version will be downloaded to search against.
- The full version of OAS can be downloaded here ().
- You can also give it the path for a custom database to search against. (See below for how to create a custom database).
- You can place the custom database in the OAS folder to have KA-Search search against both databases.

#### Regions to search with
- Default regions are the whole chain, CDRs or CDR3.
- User-defined regions can be added, as seen with the paratope search below.
- For each region, the search can either be based on exact length match or not.
- For a more specific search, the search can be focused on a specific chain and species.

In [6]:
paratope = ["107 ", "108 ","111C", "114 ","115 "]

In [7]:
searchdb = SearchDB(
    #database_path,                # Database to search. If not set, oas-aligned-tiny will be automatically downloaded and used
    allowed_chain='Heavy',         # Search against a specific chain. Default is any chain.
    allowed_species='Human',       # Search against a specific species. Default is any species.
    regions=['whole', 'cdrs', 'cdr3', paratope], # Regions to search with.
    length_matched=[False, True, True, True],    # Whether to search with length match or not.
)

-----------
### **Run search**

A search takes ~23min per sequence against all of OAS and ~2min per sequence against the small OAS.

To specify the number of closest similar sequences to keep, you can change the keep_best_n parameter.

In [8]:
%%time 
searchdb.search(
    aligned_seqs,   # Input can be a single or multiple aligned sequences at a time.
    keep_best_n=5,  # You can define how many most similar sequences to return
)

CPU times: user 31.5 s, sys: 3.82 s, total: 35.3 s
Wall time: 12.2 s


### Get N best identities

Identities of the most similar sequence for each of the regions can be fetched from the object with the bellow command.

In [9]:
searchdb.current_best_identities

array([[[0.93495935, 0.84375   , 0.8125    , 1.        ],
        [0.91935486, 0.8125    , 0.8125    , 1.        ],
        [0.91935486, 0.78125   , 0.8125    , 1.        ],
        [0.9186992 , 0.78125   , 0.8125    , 1.        ],
        [0.9186992 , 0.78125   , 0.8125    , 1.        ]],

       [[0.796748  , 0.6875    , 0.6875    , 1.        ],
        [0.796748  , 0.6875    , 0.6875    , 1.        ],
        [0.796748  , 0.6875    , 0.6875    , 1.        ],
        [0.796748  , 0.65625   , 0.6875    , 1.        ],
        [0.796748  , 0.65625   , 0.625     , 1.        ]]], dtype=float32)

---------
## Extract the meta data from matched sequences

Using the get_meta function, the meta data for all matched sequences for each query and region can be extracted as seen below.

Zero (0) is the first query or the first region in the list when initiating the search class. 

NB: The column "sequence_alignment_aa" holds the antibody sequence.

In [ ]:
n_best_sequences = searchdb.get_meta(
    n_query = 0,          # Which query to extract meta data from
    n_region = 0,         # Which region to extract meta data from
    n_sequences = 10,     # Number of sequences to extract (default is all, which is keep_best_n)
    n_jobs = 10           # Allocated number for jobs/threads for the extraction
)
n_best_sequences

In [ ]:
n_best_sequences.sequence_alignment_aa.values

----------
## Create custom database


To create your own database you first need to create a csv file in the OAS format. For an example file, look at data/custom-data-example.csv. This file consists of a dictionary containing the metadata in the first line and then rows of the individual sequences afterwards. Only the Species and Chain is strictly needed in the metadata, and only the amino acids sequence of the antibodies is required for each antibody sequence.

### 1. Format your data into OAS files

In [ ]:
import json, os, shutil
import pandas as pd

In [ ]:
custom_data_file = "custom-data-examples.csv"

seq_df = pd.DataFrame([
    ["EVQLVESGGGLAKPGGSLRLHCAASGFAFSSYWMNWVRQAPGKRLEWVSAINLGGGLTYYAASVKGRFTISRDNSKNTLSLQMNSLRAEDTAVYYCATDYCSSTYCSPVGDYWGQGVLVTVSS"],
    ["EVQLVQSGAEVKRPGESLKISCKTSGYSFTSYWISWVRQMPGKGLEWMGAIDPSDSDTRYNPSFQGQVTISADKSISTAYLQWSRLKASDTATYYCAIKKYCTGSGCRRWYFDLWGPGT"],
    ['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGEPRYDYAWFAYWGQGTLVTVS'],
    ['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGPATAWFAYWGQGTLVTVS'],
    ['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARSAWFAYWGQGTLVTVS'],
    ['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGGYWGQGTTLTVSS'],
    ['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGGLRRGAWFAYWGQGTLVTVS']
], columns = ['heavy_sequences'])
meta_data = pd.Series(name=json.dumps({"Species":"Human", "Chain":"Heavy"}), dtype='object')

meta_data.to_csv(custom_data_file, index=False)
seq_df.to_csv(custom_data_file, index=False, mode='a')

### 2. Turn your OAS formatted files into a custom database

After creating all the files you want to include in the new database, you can run the following code to create the database.

**NB:** Each csv file needs to be moved to the 'extra_data' folder, for extraction of meta data

In [ ]:
path_to_custom_db = "my_kasearch_db"
many_custom_data_files = [custom_data_file]

In [ ]:
%%timeit -n 1 -r 1

customDB = PrepareDB(db_path=path_to_custom_db, n_jobs=2, from_scratch=True)

for num, data_file in enumerate(many_custom_data_files):
    
    customDB.prepare_sequences(
        data_file,
        file_id=num, 
        chain='Heavy', 
        species='Human',
        seq_column_name = 'heavy_sequences',
    )
    shutil.copy(data_file, os.path.join(path_to_custom_db, 'extra_data'))
    
customDB.finalize_prepared_files()

### 3. Initiate the search class with your custom database


In [ ]:
raw_queries = [
    'VKLLEQSGAEVKKPGASVKVSCKASGYSFTSYGLHWVRQAPGQRLEWMGWISAGTGNTKYSQKFRGRVTFTRDTSATTAYMGLSSLRPEDTAVYYCARDPYGGGKSEFDYWGQGTLVTVSS',
]

results = EasySearch(
    raw_queries, 
    keep_best_n=10,
    database_path=path_to_custom_db, 
    allowed_chain='Any', 
    allowed_species='Any',
    regions=['cdr3'],
    length_matched=[False],
)
results